# Pick a Sudoku Puzzle

## Load Sudoku Webpage

In [1]:
import requests 
from bs4 import BeautifulSoup

vgm_url = 'http://lipas.uwasa.fi/~timan/sudoku/'
html_text = requests.get(vgm_url).text
soup = BeautifulSoup(html_text, 'html.parser')

## Pick a Level

In [77]:
lvl = input('Pick a level from 1 to 15: ')
pzl = input('Pick a puzzle; a, b, or c: ')
pzFile = 's' + str(lvl).zfill(2) + pzl + '.txt'
print(pzFile)

Pick a level from 1 to 15: 5
Pick a puzzle; a, b, or c: a
s05a.txt


## Parse Webpage

### Get link for .txt file

In [78]:
for tx in soup.find_all('a'):
#     print(tx)
#     print(tx.string)
    if tx.string == pzFile:
        lnk = vgm_url+"/"+tx['href']
        print(lnk)
        
#     print('---------------------')

http://lipas.uwasa.fi/~timan/sudoku//s05a.txt


### Read data from .txt file

In [79]:
import urllib
import pandas as pd

f = urllib.request.urlopen(lnk)
myfile = f.readlines()
sdk = []
#print(myfile)

for i, line in enumerate(myfile):
    sdk.append([])
    for j, value in enumerate(line.split()):
        sdk[i].append(int(value))

df = pd.DataFrame(sdk)
if len(sdk) == 10:
    df.drop([9], inplace=True)
    del sdk[9]
    
print(sdk)
print(df)

[[1, 0, 5, 0, 0, 0, 3, 7, 0], [0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 9, 7, 3, 0, 0, 0, 1, 0], [0, 0, 0, 0, 5, 3, 1, 0, 2], [3, 0, 0, 8, 0, 1, 0, 0, 4], [2, 0, 1, 4, 7, 0, 0, 0, 0], [0, 7, 0, 0, 0, 8, 6, 4, 0], [0, 0, 8, 0, 0, 0, 0, 0, 0], [0, 1, 2, 0, 0, 0, 8, 0, 7]]
     0    1    2    3    4    5    6    7    8
0  1.0  0.0  5.0  0.0  0.0  0.0  3.0  7.0  0.0
1  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0
2  0.0  9.0  7.0  3.0  0.0  0.0  0.0  1.0  0.0
3  0.0  0.0  0.0  0.0  5.0  3.0  1.0  0.0  2.0
4  3.0  0.0  0.0  8.0  0.0  1.0  0.0  0.0  4.0
5  2.0  0.0  1.0  4.0  7.0  0.0  0.0  0.0  0.0
6  0.0  7.0  0.0  0.0  0.0  8.0  6.0  4.0  0.0
7  0.0  0.0  8.0  0.0  0.0  0.0  0.0  0.0  0.0
8  0.0  1.0  2.0  0.0  0.0  0.0  8.0  0.0  7.0


# Solve the Puzzle

## Get available numbers for each cell

In [5]:
import math

def getAvailable(sudoku):
    rows = [[0 for i in range(9)] for j in range(9)] 
    cols = [[0 for i in range(9)] for j in range(9)] 
    sqrs = [[0 for i in range(9)] for j in range(9)] 

    for i in range(9):
        for j in range(9):
            sqNum = math.floor(i/3.0)*3 + math.floor(j/3.0)
            if sudoku[i][j] > 0:
                rows[i][sudoku[i][j]-1] += 1
                cols[j][sudoku[i][j]-1] += 1
                sqrs[sqNum][sudoku[i][j]-1] += 1
                
    cellOpt = [[0 for i in range(9)] for j in range(9*9)] 
    for i in range(9*9):
        rw = math.floor(i/9.0)
        cl = int(i%9.0)
        sqNum = math.floor(rw/3.0)*3 + math.floor(cl/3.0)
    #     print(str(rw) + " " + str(cl) + " " + str(sqNum) + " " + str(i) + " " + str(j))    
        for j in range(9):
    #         print("i:" + str(i) + " j:" + str(j) + " " + str(rows[rw][j]) + " " + str(rows[cl][j]) + " " + str(sqrs[sqNum][j]))
            if rows[rw][j] + cols[cl][j] + sqrs[sqNum][j] == 0 and sudoku[rw][cl] == 0:
                cellOpt[i][j] = 1
#     print(np.matrix(cellOpt))
    return cellOpt, rows, cols, sqrs

## Solve the puzzle

In [95]:
import numpy as np
import copy
import time

def solve(sudoku, recLevel=0, outputFormat='none'):
    sud = copy.deepcopy(sudoku)
    solved = 0
    clFound = 1
    if outputFormat == 'verbose': print('Recursion Level %d' % (recLevel))
    if outputFormat == 'minimal': 
        print('Recursion Level %d                                           ' % (recLevel), end='\r')
        time.sleep(0.5)
    # Find explicit solutions
    while clFound == 1:
        clFound = 0 
        cellAvail, rows, cols, sqrs = getAvailable(sud)
        for i in range(9*9):
            if np.sum(cellAvail[i]) == 1:
                rw = math.floor(i/9.0)
                cl = int(i%9.0)
                sqNum = math.floor(rw/3.0)*3 + math.floor(cl/3.0)
                l='Row Options:    '
                m='Column Options: '
                n='Square Options: '
                for a in rows[rw]:
                    l += str(a) + ' '
                if outputFormat == 'verbose': print(l)

                for b in cols[cl]:
                    m += str(b) + ' '
                if outputFormat == 'verbose': print(m)

                for c in sqrs[sqNum]:
                    n += str(c) + ' '
                if outputFormat == 'verbose': print(n)

                if outputFormat == 'verbose': 
                    print("row: %d  column: %d" % (rw,cl))
                    print("available " + str(cellAvail[i]))
                    print("Number is %d" % (cellAvail[i].index(1) + 1))
                sud[rw][cl] = cellAvail[i].index(1) + 1
                clFound = 1
    
    if outputFormat == 'verbose': 
        print('Implicit Solution at Recursion Level %d' % (recLevel))
        print(printSudoku(sud))
    
    cellAvail, rows, cols, sqrs = getAvailable(sud)
    numAvail = [max(x, y, z) for x, y, z in zip(rows, cols, sqrs)]
    if any(v > 1 for ln in numAvail for v in ln):
        solved = 0
        if outputFormat == 'verbose': print('not solved, exit at line 41')
        return solved, sud, recLevel
        
    # Check if puzzle is solved
    if not any(0 in ln for ln in sud):
        solved = 1
        if outputFormat == 'verbose': print("Solved at line 45")
        return solved, sud, recLevel
    
    # Test implicit solutions
    for i in range(9*9):
        if np.sum(cellAvail[i]) > 0:
            for j in range(9):
                if cellAvail[i][j] == 1:
                    rw = math.floor(i/9.0)
                    cl = int(i%9.0)
                    tsol = copy.deepcopy(sud)
                    if outputFormat == 'verbose': print('Testing value %d in location (%d, %d)' % (j+1, rw, cl))
                    if outputFormat == 'minimal': 
                        print('Recursion Level %d - Testing value %d in location (%d, %d)' % (recLevel, j+1, rw, cl), end='\r') 
                        time.sleep(0.5)
                    tsol[rw][cl] = j + 1
                    recLevel += 1
                    solved, tsol, recLevel = solve(tsol, recLevel, outputFormat)
                    if outputFormat == 'minimal': 
                        print('Recursion Level %d                                           ' % (recLevel), end='\r')
                        time.sleep(0.5)
                    recLevel -= 1
                    if solved == 1: 
                        if outputFormat == 'verbose': print("Solved at line 62")
                        sud = copy.deepcopy(tsol)
                        return solved, sud, recLevel
    
    
    if not any(0 in ln for ln in sud) or solved == 1:
        solved = 1
        if outputFormat == 'verbose': print("Solved at line 69")
        return solved, sud, recLevel
    else:
        if outputFormat == 'verbose': print("not solved, exit at line 72")
        solved = 0
        return solved, sud, recLevel

In [96]:
svld, sol, recL = solve(sdk,outputFormat='minimal')

# Print Solution

In [45]:
def printSudoku(sud):
    df = pd.DataFrame(sud)
    df.insert(0,9,['|']*9)
    df.insert(10,10,['|']*9)
    df.insert(4,11,['|']*9)
    df.insert(8,12,['|']*9)

    df.columns = range(df.shape[1])
    df.reset_index()
    df1 = pd.DataFrame([['-']*13])
    sudFormat = pd.concat([df1, df[0:3], df1, df[3:6], df1, df[6:9], df1])

    return sudFormat

In [76]:
print(printSudoku(sol))

  0  1  2  3  4  5  6  7  8  9  10 11 12
0  -  -  -  -  -  -  -  -  -  -  -  -  -
0  |  1  2  5  |  6  4  9  |  3  7  8  |
1  |  8  3  4  |  7  1  5  |  2  9  6  |
2  |  6  9  7  |  3  8  2  |  4  1  5  |
0  -  -  -  -  -  -  -  -  -  -  -  -  -
3  |  7  4  6  |  9  5  3  |  1  8  2  |
4  |  3  5  9  |  8  2  1  |  7  6  4  |
5  |  2  8  1  |  4  7  6  |  9  5  3  |
0  -  -  -  -  -  -  -  -  -  -  -  -  -
6  |  5  7  3  |  2  9  8  |  6  4  1  |
7  |  4  6  8  |  1  3  7  |  5  2  9  |
8  |  9  1  2  |  5  6  4  |  8  3  7  |
0  -  -  -  -  -  -  -  -  -  -  -  -  -
